In [9]:
from hamcrest import equal_to_ignoring_whitespace
import psycopg2
import pandas as pd
from flask import Flask, jsonify
import sqlite3

# Connect to gun_violence database
con = sqlite3.connect("gun_violence.sqlite")

# Read data from gunviolencedata table and load into a DataFrame instance
raw_data = pd.read_sql("SELECT date, city_or_county, state, n_killed, n_injured, latitude, longitude, population from gunviolencedata WHERE (n_killed > 0 OR n_injured > 0);", con);
 
pd.set_option('display.expand_frame_repr', False);

# Print the DataFrame
con.close()
print(raw_data);

           date city_or_county           state  n_killed  n_injured  latitude  longitude population
0        1/1/13     Mckeesport    Pennsylvania         0          4   40.3467   -79.8559       None
1        1/1/13      Hawthorne      California         1          3   33.9090  -118.3330       None
2        1/1/13         Lorain            Ohio         1          3   41.4455   -82.1377       None
3        1/5/13         Aurora        Colorado         4          0   39.6518  -104.8020       None
4        1/7/13     Greensboro  North Carolina         2          2   36.1140   -79.9569       None
...         ...            ...             ...       ...        ...       ...        ...        ...
142385  3/31/18        Chicago        Illinois         0          1       NaN        NaN       None
142386  3/31/18   Natchitoches       Louisiana         1          0   31.7537   -93.0836       None
142387  3/31/18         Gretna       Louisiana         0          1   29.9239   -90.0442       None


In [7]:
raw_data['date'] = pd.to_datetime(raw_data['date'])
# Create a new column with value of 1
raw_data['incident_count'] = 1

,date,city_or_county,state,n_killed,n_injured,latitude,longitude,population,incident_count
0,2013-01-01,Mckeesport,Pennsylvania,0,4,40.3467,-79.8559,None,1
1,2013-01-01,Hawthorne,California,1,3,33.9090,-118.3330,None,1
2,2013-01-01,Lorain,Ohio,1,3,41.4455,-82.1377,None,1
3,2013-01-05,Aurora,Colorado,4,0,39.6518,-104.8020,None,1
4,2013-01-07,Greensboro,North Carolina,2,2,36.1140,-79.9569,None,1
...,...,...,...,...,...,...,...,...,...
142385,2018-03-31,Chicago,Illinois,0,1,NaN,NaN,None,1
142386,2018-03-31,Natchitoches,Louisiana,1,0,31.7537,-93.0836,None,1
142387,2018-03-31,Gretna,Louisiana,0,1,29.9239,-90.0442,None,1
142388,2018-03-31,Houston,Texas,1,0,29.7201,-95.6110,None,1


In [8]:

grouped_series = raw_data.groupby(['state', 'city_or_county', 'population'])['incident_count'].sum().sort_values(ascending=False)
grouped_series


Series([], Name: incident_count, dtype: int64)

In [4]:
# Create a dataframe from series. Rename one of the indexes

grouped_df = grouped_series.to_frame()
grouped_df.index = grouped_df.index.set_names('city', level=1)
grouped_df.head()

,,,incident_count
state,city,population,
Maryland,Baltimore,585708,2794
Pennsylvania,Philadelphia,1603797,2433
Louisiana,New Orleans,383997,1938
Tennessee,Memphis,632207,1732
Texas,Houston,2302792,1688


In [5]:
grouped_df['per_capita'] = grouped_df.eval('(incident_count/population)*100000')
grouped_df.sort_values(by='per_capita', ascending=False)



,,,incident_count,per_capita
state,city,population,,
Delaware,Wilmington,70941,554,780.930632
Indiana,Gary,68982,378,547.969035
Michigan,Flint,81381,419,514.862191
Louisiana,New Orleans,383997,1938,504.691443
Mississippi,Meridian,35001,173,494.271592
...,...,...,...,...
California,San Bernardino County,2181654,1,0.045837
Arizona,Maricopa County,4420568,2,0.045243
Texas,Dallas County,2613539,1,0.038262


In [6]:
# Move 'population' out of index as it was never needed there
grouped_df1 = grouped_df.reset_index(level='population')
grouped_df1.head()

,,population,incident_count,per_capita
state,city,,,
Maryland,Baltimore,585708,2794,477.029510
Pennsylvania,Philadelphia,1603797,2433,151.702491
Louisiana,New Orleans,383997,1938,504.691443
Tennessee,Memphis,632207,1732,273.960902
Texas,Houston,2302792,1688,73.302322


In [7]:
cities= pd.read_csv('uscities.csv')
# cities.reset_index()
# cities.set_index(['state_name', 'city'], inplace=True)
# cities.index = cities.index.set_names('state', level=0)
cities.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36081,Queens,40.6943,-73.9249,18680025,10768.0,shape,False,True,America/New_York,1,11229 11228 11226 11225 11224 11222 11221 1122...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,12531334,3267.0,shape,False,True,America/Los_Angeles,1,91367 90291 90293 90292 91316 91311 90035 9003...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8586888,4576.0,shape,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,6076316,4945.0,shape,False,True,America/New_York,1,33128 33129 33125 33126 33127 33149 33144 3314...,1840015149
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7935,-96.7667,5910669,1522.0,shape,False,True,America/Chicago,1,75098 75287 75230 75231 75236 75237 75235 7525...,1840019440


In [8]:
coord = cities[['state_name','city', 'lat', 'lng']]
coord.reset_index()
coord.set_index(['state_name', 'city'], inplace=True)
coord.index = coord.index.set_names('state', level=0)
coord


,,lat,lng
state,city,,
New York,New York,40.6943,-73.9249
California,Los Angeles,34.1141,-118.4068
Illinois,Chicago,41.8375,-87.6866
Florida,Miami,25.7840,-80.2101
Texas,Dallas,32.7935,-96.7667
...,...,...,...
Idaho,Drummond,43.9996,-111.3433
Wyoming,Lost Springs,42.7652,-104.9255
South Dakota,Provo,43.1937,-103.8329


In [10]:
grouped_df1.head()


,,population,incident_count,per_capita
state,city,,,
Maryland,Baltimore,585708,2794,477.029510
Pennsylvania,Philadelphia,1603797,2433,151.702491
Louisiana,New Orleans,383997,1938,504.691443
Tennessee,Memphis,632207,1732,273.960902
Texas,Houston,2302792,1688,73.302322


In [11]:


result = pd.merge(grouped_df1, coord, how='left', on=['state', 'city'])
result.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2219 entries, ('Maryland', 'Baltimore') to ('Wyoming', 'Sheridan County')
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   population      2219 non-null   int64  
 1   incident_count  2219 non-null   int64  
 2   per_capita      2219 non-null   float64
 3   lat             1322 non-null   float64
 4   lng             1322 non-null   float64
dtypes: float64(3), int64(2)
memory usage: 174.9+ KB


In [12]:
result.to_csv('us_per_capita.csv')
result = result.sort_values(by='per_capita', ascending=False)
result.head(50)

population  incident_count  per_capita      lat      lng
state          city                                                                     
Delaware       Wilmington            70941             554  780.930632  39.7415 -75.5416
Indiana        Gary                  68982             378  547.969035  41.5905 -87.3473
Michigan       Flint                 81381             419  514.862191  43.0236 -83.6921
Louisiana      New Orleans          383997            1938  504.691443  30.0687 -89.9288
Mississippi    Meridian              35001             173  494.271592  32.3846 -88.6896
New Jersey     Trenton               90857             437  480.975599  40.2237 -74.7641
Maryland       Baltimore            585708            2794  477.029510  39.3051 -76.6144
Pennsylvania   Harrisburg            50090             214  427.230984  40.2752 -76.8843
               Chester               32718             139  424.842594  39.8456 -75.3718
               York                  44810             180  401.696050  39.9651 -76.7315
South Carolina Myrtle Beach          35868             143  398.684064  33.7094 -78.8844
Mississippi    Jackson              153643             603  392.468254  32.3157 -90.2125
South Carolina Anderson              29170             114  390.812479  34.5211 -82.6478
Michigan       Saginaw               44166             167  378.118915  43.4199 -83.9501
Alabama        Birmingham           200763             751  374.072912  33.5279 -86.7971
Georgia        Savannah             148095             538  363.280327  32.0286 -81.1821
Ohio           Cleveland            373091            1294  346.832274  41.4764 -81.6805
North Carolina Goldsboro             33107             111  335.276528  35.3778 -77.9717
Virginia       Richmond             226610             731  322.580645  37.5295 -77.4756
New Jersey     Atlantic City         38504             124  322.044463  39.3797 -74.4527
South Carolina Spartanburg           38567             117  303.368164  34.9442 -81.9251
Virginia       Petersburg            33458             100  298.882181  37.2043 -77.3913
Louisiana      Shreveport           187993             549  292.032150  32.4653 -93.7955
Arkansas       Little Rock          202514             575  283.930987  34.7256 -92.3577
Tennessee      Chattanooga          181234             514  283.611243  35.0660 -85.2481
Louisiana      Baton Rouge          225128             622  276.287268  30.4420 -91.1311
Tennessee      Memphis              632207            1732  273.960902  35.1087 -89.9663
South Carolina Aiken                 31800              87  273.584906  33.5303 -81.7271
West Virginia  Charleston            48913             127  259.644675  38.3484 -81.6322
Indiana        East Chicago          26314              68  258.417572  41.6469 -87.4545
Ohio           Cincinnati           310242             801  258.185545  39.1413 -84.5060
Alabama        Bessemer              26044              67  257.256950  33.3712 -86.9728
New York       Syracuse             147363             377  255.830839  43.0407 -76.1437
Florida        West Palm Beach      117151             294  250.958165  26.7469 -80.1320
Arkansas       Pine Bluff            41246             103  249.721185  34.2116 -92.0173
South Carolina Columbia             137300             342  249.089585  34.0378 -80.9036
North Carolina Salisbury             35567              87  244.608767  35.6658 -80.4905
Virginia       Danville              42590             104  244.188777  36.5831 -79.4088
Illinois       Rockford             149009             363  243.609446  42.2596 -89.0640
Delaware       Dover                 39614              96  242.338567  39.1610 -75.5202
South Carolina Greenville            70633             167  236.433395  34.8354 -82.3646
Illinois       Peoria               113173             267  235.921996  40.7520 -89.6154
New Jersey     Camden                72001             168  233.330093  39.9361 -75.1073
Tennessee      Jackson     

In [13]:

california = result.query("state == 'California'")
california = california.sort_values(by='per_capita', ascending=False)
california.head(20)

population  incident_count  per_capita      lat       lng
state      city                                                                         
California Oakland                 439349             810  184.363684  37.7904 -122.2166
           Eureka                   26547              37  139.375447  40.7943 -124.1564
           Desert Hot Springs       32529              45  138.338098  33.9550 -116.5429
           Salinas                 163687             198  120.962569  36.6883 -121.6317
           Richmond                116287             115   98.893255  37.9477 -122.3389
           Compton                  95959              91   94.832168  33.8930 -118.2274
           Fresno                  542161             495   91.301292  36.7830 -119.7939
           San Bernardino          221898             194   87.427557  34.1416 -117.2943
           Antioch                 115360              98   84.951456  37.9787 -121.7960
           Stockton                320759             266   82.928304  37.9765 -121.3109
           Bakersfield             402907             319   79.174599  35.3529 -119.0359
           Merced                   86141              53   61.527031  37.3057 -120.4779
           Sacramento              522754             318   60.831672  38.5677 -121.4685
           Norwalk                 102910              59   57.331649  33.9069 -118.0829
           Atwater                  31819              18   56.569974  37.3529 -120.5959
           Oxnard                  202185             106   52.427232  34.1964 -119.1815
           Watsonville              52739              27   51.195510  36.9206 -121.7708
           Pomona                  151554              77   50.806973  34.0585 -117.7626
           Vallejo                 126035              63   49.986115  38.1125 -122.2342
           Long Beach              466302             233   49.967618  33.7977 -118.1670

In [14]:
california.to_csv('ca_per_capita.csv')
california.head()

population  incident_count  per_capita      lat       lng
state      city                                                                         
California Oakland                 439349             810  184.363684  37.7904 -122.2166
           Eureka                   26547              37  139.375447  40.7943 -124.1564
           Desert Hot Springs       32529              45  138.338098  33.9550 -116.5429
           Salinas                 163687             198  120.962569  36.6883 -121.6317
           Richmond                116287             115   98.893255  37.9477 -122.3389